### 第一步：将特征完全相同的产品归并

In [ ]:
import pandas as pd
input_file = "../productLabels_multiSpreadsheets.xlsx"
output_file = "./first_cluster.xlsx"
product_column = "prod_id"


In [18]:
# 读取xlsx文件
all_sheets = pd.read_excel(input_file, sheet_name=None)
# 2. 创建一个空字典，用于存储每个sheet处理后的结果
grouped_results = {}
# 3. 逐个sheet处理
for sheet_name, df in all_sheets.items():
    print(f"正在处理 sheet：{sheet_name} ...")
    # 检查产品名称列是否存在
    if product_column not in df.columns:
        print(f"⚠️ 跳过 {sheet_name}：未找到列 '{product_column}'")
        continue
    # 获取特征列（除产品列外）
    feature_cols = [col for col in df.columns if col != product_column]
    # 按特征列分组，收集产品列表
    grouped = (
        df.groupby(feature_cols, dropna=False)[product_column]
        .apply(list)
        .reset_index()
        .rename(columns={product_column: "prod_id_list"})
    )
    # 调整列顺序 & 按第一个产品排序
    cols = ["prod_id_list"] + [c for c in grouped.columns if c != "prod_id_list"]
    grouped = grouped[cols]
    grouped = grouped.sort_values(by="prod_id_list", key=lambda x: x.str[0])
    # 归并后形成新的第一步聚类后的id(first_cluster_id)
    grouped["first_cluster_id"] = [str(sheet_name)+ str(idx) for idx, _ in enumerate(grouped["prod_id_list"])]
    # 存入结果字典
    grouped_results[sheet_name] = grouped

# 4. 将所有结果写入新的Excel文件
with pd.ExcelWriter(output_file) as writer:
    for sheet_name, grouped_df in grouped_results.items():
        grouped_df.to_excel(writer, sheet_name=sheet_name, index=False)

print(f"✅ 全部处理完成！结果已保存为：{output_file}")

正在处理 sheet：D ...
正在处理 sheet：C ...
正在处理 sheet：A ...
正在处理 sheet：N ...
正在处理 sheet：P ...
✅ 全部处理完成！结果已保存为：./first_cluster.xlsx


### 第二步：合并event_dataset和cust_dataset, 并将D、C、A、N、P数据分列在不同的sheet

In [19]:
import numpy as np

# 读取客户文件
cust_dataset = pd.read_csv('../cust_dataset.csv')
cust_dataset = pd.DataFrame(cust_dataset)

# 读取事件文件
event_dataset = pd.read_csv('../event_dataset.csv')
event_dataset = pd.DataFrame(event_dataset)

print("cust_dataset shape: ", cust_dataset.shape)
print("event_dataset shape: ", event_dataset.shape)

cust_dataset shape:  (1108827, 7)
event_dataset shape:  (338939, 9)


In [20]:
# 读取产品文件
excel_file = pd.ExcelFile('../productLabels_multiSpreadsheets.xlsx', engine='openpyxl') 
all_sheets = pd.read_excel(excel_file, sheet_name=None) 
# print("工作表名称：", list(all_sheets.keys()))
D_dataset = pd.DataFrame(all_sheets['D'])
D_dataset.columns = [col if col == 'prod_id' else f'D_{col}' for col in D_dataset.columns]

C_dataset = pd.DataFrame(all_sheets['C'])
C_dataset.columns = [col if col == 'prod_id' else f'C_{col}' for col in C_dataset.columns]

A_dataset = pd.DataFrame(all_sheets['A'])
A_dataset.columns = [col if col == 'prod_id' else f'A_{col}' for col in A_dataset.columns]

N_dataset = pd.DataFrame(all_sheets['N'])
N_dataset.columns = [col if col == 'prod_id' else f'N_{col}' for col in N_dataset.columns]

P_dataset = pd.DataFrame(all_sheets['P'])
P_dataset.columns = [col if col == 'prod_id' else f'P_{col}' for col in P_dataset.columns]

In [21]:
# 数据连接：我们以事件表为核心，增加了客户、产品的信息
event_cust = pd.merge(event_dataset,cust_dataset, on='cust_no', how = 'left')
event_cust = pd.merge(event_cust,D_dataset,on = 'prod_id',how = 'left')
event_cust = pd.merge(event_cust,C_dataset,on = 'prod_id',how = 'left')
event_cust = pd.merge(event_cust,N_dataset,on = 'prod_id',how = 'left')
event_cust = pd.merge(event_cust,A_dataset,on = 'prod_id',how = 'left')
event_cust = pd.merge(event_cust,P_dataset,on = 'prod_id',how = 'left')
event_cust.to_csv("event_cust.csv")

In [26]:
# 筛选 prod_id 以 "sheet_name" 开头的行
for sheet_name, grouped_df in grouped_results.items():
    filtered_df = event_cust[event_cust['prod_id'].astype(str).str.startswith(sheet_name)]

    # 为快速查找，先把 grouped 转换为字典映射：prod_id → first_cluster_id
    mapping = {}

    for _, row in grouped_df.iterrows():
        for pid in row['prod_id_list']:
            mapping[pid] = row['first_cluster_id']
            
    # 在 filtered_df 中生成新列 first_cluster_id
    filtered_df['first_cluster_id'] = filtered_df['prod_id'].map(mapping)
    filtered_df.to_csv("event_cust_"+str(sheet_name)+".csv")
    print(sheet_name,":\n",filtered_df["first_cluster_id"].value_counts())

C:\Users\肖异芳\AppData\Local\Temp\ipykernel_23520\1219157190.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['first_cluster_id'] = filtered_df['prod_id'].map(mapping)


D :
 first_cluster_id
D0     76163
D1     65822
D2     43557
D3     29790
D4     29045
D5      8310
D6      1100
D8        63
D7        45
D9        32
D10       19
D11        3
Name: count, dtype: int64


C:\Users\肖异芳\AppData\Local\Temp\ipykernel_23520\1219157190.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['first_cluster_id'] = filtered_df['prod_id'].map(mapping)


C :
 first_cluster_id
C0     18246
C6      7292
C2      5982
C7      5270
C22     3051
C3       957
C11      839
C12      709
C15      515
C1       381
C10      306
C5       242
C4       221
C13      209
C14      162
C17       64
C9        63
C19       26
C21       24
C8        14
C18       13
C25       11
C16       10
C29        9
C28        6
C26        5
C20        5
C23        2
C27        1
C24        1
C30        1
Name: count, dtype: int64
A :
 first_cluster_id
A1    1855
A0    1160
A2     657
A6     556
A5     166
A3      82
A7      70
A4      29
Name: count, dtype: int64


C:\Users\肖异芳\AppData\Local\Temp\ipykernel_23520\1219157190.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['first_cluster_id'] = filtered_df['prod_id'].map(mapping)
C:\Users\肖异芳\AppData\Local\Temp\ipykernel_23520\1219157190.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['first_cluster_id'] = filtered_df['prod_id'].map(mapping)


N :
 first_cluster_id
N0    21486
N1       13
Name: count, dtype: int64
P :
 first_cluster_id
P0    8788
P2     338
P1      97
Name: count, dtype: int64


C:\Users\肖异芳\AppData\Local\Temp\ipykernel_23520\1219157190.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['first_cluster_id'] = filtered_df['prod_id'].map(mapping)
